In [1]:
import keras
import tensorflow as tf
print(keras.__version__)
print(tf.__version__)

2.5.0
2.5.1


In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

NGRAMS = 2
SAMPLE = 1000000
EPOCHS = 20

# Florida voter
df = pd.read_csv('../dataverse_files/fl_voter_reg/fl_reg_name_race.csv.gz')
df.dropna(subset=['name_first', 'name_last'], inplace=True)
df['race'] = df.race.map({'native_indian': 'other', 'asian': 'asian', 'nh_black': 'nh_black', 'hispanic': 'hispanic', 'nh_white': 'nh_white', 'other': 'other', 'multi_racial': 'other', 'unknown': 'unknown'})
df

,name_last,name_first,race
0,Walker,Elizabeth,nh_white
1,Palmer,Alton,nh_white
2,Mc Cleod,Alicia,nh_black
3,Scarborough,Dale,nh_white
4,Walker,Daniel,nh_white
...,...,...,...
13653889,Walters,William,nh_white
13653890,Sawyer,Matthew,nh_white
13653891,Thomas,Janine,nh_white
13653892,Campbell,Angel,other


In [3]:
sdf = df[df.race.isin(['unknown']) == False].groupby(['race']).sample(int(SAMPLE/5), random_state=21)
del df

# Additional features
sdf['name_last'] = sdf.name_last.str.title()
sdf

,name_last,name_first,race
779856,Kelley,Mi,asian
10722411,Pitman,Ariel,asian
9871739,Patel,RANJANBALA,asian
6278811,Inch,Corazon,asian
6085057,Mathew,Lilly,asian
...,...,...,...
5566342,Rogers,Joanne,other
52018,Akther,Nasrin,other
2691413,Menendez,Rafaela,other
8309012,Hegdal,Gertie,other


In [4]:
rdf = sdf.groupby('race').agg({'name_last': 'count'})
rdf.to_csv('../dataverse_files/fl_voter_reg/lstm/fl_ln_five_cat_race.csv', columns=[])
rdf

,name_last
race,
asian,200000
hispanic,200000
nh_black,200000
nh_white,200000
other,200000


In [5]:
sdf.groupby('race').agg({'name_last': 'nunique'})

,name_last
race,
asian,49319
hispanic,47360
nh_black,28405
nh_white,65081
other,65474


## Preprocessing the input data

In [6]:
# last name only
sdf['name_last_name_first'] = sdf['name_last']

# build n-gram list
vect = CountVectorizer(analyzer='char', max_df=0.3, min_df=3, ngram_range=(NGRAMS, NGRAMS), lowercase=False) 
a = vect.fit_transform(sdf.name_last_name_first)
vocab = vect.vocabulary_

# sort n-gram by freq (highest -> lowest)
words = []
for b in vocab:
    c = vocab[b]
    #print(b, c, a[:, c].sum())
    words.append((a[:, c].sum(), b))
    #break
words = sorted(words, reverse=True)
words_list = [w[1] for w in words]
num_words = len(words_list)
print("num_words = %d" % num_words)


def find_ngrams(text, n):
    a = zip(*[text[i:] for i in range(n)])
    wi = []
    for i in a:
        w = ''.join(i)
        try:
            idx = words_list.index(w)
        except:
            idx = 0
        wi.append(idx)
    return wi

# build X from index of n-gram sequence
X = np.array(sdf.name_last_name_first.apply(lambda c: find_ngrams(c, NGRAMS)))

# check max/avg feature
X_len = []
for x in X:
    X_len.append(len(x))

max_feature_len = max(X_len)
avg_feature_len = int(np.mean(X_len))

print("Max feature len = %d, Avg. feature len = %d" % (max_feature_len, avg_feature_len))
y = np.array(sdf.race.astype('category').cat.codes)

# Split train and test dataset
X_train,  X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

num_words = 1196
Max feature len = 27, Avg. feature len = 5


## Train a LSTM model

ref: http://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/

In [7]:
'''The dataset is actually too small for LSTM to be of any advantage
compared to simpler, much faster methods such as TF-IDF + LogReg.
Notes:

- RNNs are tricky. Choice of batch size is important,
choice of loss and optimizer is critical, etc.
Some configurations won't converge.

- LSTM loss decrease patterns during training can be quite different
from what you see with CNNs/MLPs/etc.
'''
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, Activation
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import load_model

max_features = num_words # 20000
feature_len = 20 # avg_feature_len # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=feature_len)
X_test = sequence.pad_sequences(X_test, maxlen=feature_len)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

800000 train sequences
200000 test sequences
Pad sequences (samples x time)
X_train shape: (800000, 20)
X_test shape: (200000, 20)
5 classes
Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (800000, 5)
y_test shape: (200000, 5)


In [8]:
print('Build model...')

input_ = keras.layers.Input(shape=(feature_len))
layer_ = Embedding(num_words, 32) (input_)
layer_ = LSTM(128, dropout=0.2, recurrent_dropout=0.2)(layer_, training=True)
output = Dense(num_classes, activation='softmax') (layer_)

model = keras.models.Model(inputs=input_, outputs=output)

# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

Build model...
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 20)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 20, 32)            38272     
_________________________________________________________________
lstm (LSTM)                  (None, 128)               82432     
_________________________________________________________________
dense (Dense)                (None, 5)                 645       
Total params: 121,349
Trainable params: 121,349
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
print('Train...')
model.fit(X_train, y_train, batch_size=batch_size, epochs=EPOCHS,
          validation_split=0.1, verbose=1)
score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size, verbose=1)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Epoch 1/20
22500/22500 [==============================] - 461s 19ms/step - loss: 1.2504 - accuracy: 0.4802 - val_loss: 1.1722 - val_accuracy: 0.5212
Epoch 2/20
22500/22500 [==============================] - 565s 25ms/step - loss: 1.1624 - accuracy: 0.5278 - val_loss: 1.1498 - val_accuracy: 0.5339
Epoch 3/20
22500/22500 [==============================] - 813s 36ms/step - loss: 1.1389 - accuracy: 0.5405 - val_loss: 1.1382 - val_accuracy: 0.5418
Epoch 4/20
22500/22500 [==============================] - 813s 36ms/step - loss: 1.1242 - accuracy: 0.5489 - val_loss: 1.1322 - val_accuracy: 0.5455
Epoch 5/20
22500/22500 [==============================] - 775s 34ms/step - loss: 1.1171 - accuracy: 0.5525 - val_loss: 1.1287 - val_accuracy: 0.5472
Epoch 6/20
22500/22500 [==============================] - 772s 34ms/step - loss: 1.1135 - accuracy: 0.5554 - val_loss: 1.1258 - val_accuracy: 0.5503
Epoch 7/20
22500/22500 [==============================] - 779s 35ms/step - loss: 1.1106 - accurac

In [10]:
print('Test score:', score)
print('Test accuracy:', acc)

Test score: 1.1147631406784058
Test accuracy: 0.5564050078392029


## Confusion Matrix

In [ ]:
p = model.predict(X_test, verbose=2) # to predict probability
y_pred = np.argmax(p, axis=-1)
target_names = list(sdf.race.astype('category').cat.categories)
print(classification_report(np.argmax(y_test, axis=1), y_pred, target_names=target_names))
print(confusion_matrix(np.argmax(y_test, axis=1), y_pred))

## Save model

In [ ]:
model.save('../models/fl_voter_reg/lstm/fl_all_ln_lstm_5_cat.h5')

In [ ]:
words_df = pd.DataFrame(words_list, columns=['vocab'])
words_df.to_csv('../dataverse_files//fl_voter_reg/lstm/fl_all_ln_vocab_5_cat.csv', index=False, encoding='utf-8')